In [ ]:
!pip install cellpose==2.2 --quiet

In [ ]:
# @title Imports

!git clone https://github.com/WinetraubLab/coregister-xy.git
%cd coregister-xy

from cell_to_cell_registration import segment_cells
from cell_to_cell_registration import PCR99a
from plane.fit_plane_elastic import FitPlaneElastic

from scipy.interpolate import RegularGridInterpolator
from scipy.spatial import cKDTree
from skimage.measure import regionprops, label
from cellpose import io, models, plot

import matplotlib.pyplot as plt
import cv2
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Inputs

# Image paths
oct_volume_image_path = ""      # Path to OCT volume
hist_image_path = ""            # Path to fluorescent image

# For rough alignment
histology_coords_px = []        # Coordinates in px from fluorescent image
oct_coords_mm = []              # Matching coordinates in mm from photobleach template
smoothing = 0.5                 # 0 for fully elastic, 1 for fully affine
x_range_mm = [0, 1]             # OCT view range in x
y_range_mm = [0, 1]             # OCT view range in y

# Cell-level
do_cell_to_cell_alignment = True

# For segmentation
z_buffer_um = 20                # Desired amount of extra space above/below the rough fit estimated from the above provided coordinate pairs.
barcode_focus_oct_slice = 120   # Voxel depth/slice number in OCT volume where photobleach barcode z=0.
oct_xy_downscale_factor = 2     # Optional: Downsampling to increase segmentation speed
avg_cell_diameter_px = 10       # Expected average diameter of the cells to segment, in the raw volume (no downsampling)
hist_valid_cell_segmentation_path = None # Optional: binary mask for histology image to denote areas to keep/discard cell segmentations

# For point matching
initial_pairs_dist_threshold = 40 # Approximate accuracy of rough alignment
z_weight = 1.5                  # Increase this value the more confident you are in your z depth estimations from barcodes.